In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = "platform"

import jax
from jax import jit

from genjax import ChoiceMapBuilder as C
from genjax import (
    Diff,
    NoChange,
    UnknownChange,
    bernoulli,
    beta,
    gen,
    pretty,
)
key = jax.random.key(0)
pretty()

In [2]:
@gen
def beta_bernoulli_process(x):
    u = x + 1
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    z = 2*v
    return {'u': u, 'p': p, 'v': v, 'z': z}

In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = "platform"

import jax
from jax import jit

from genjax import ChoiceMapBuilder as C
from genjax import (
    Diff,
    NoChange,
    UnknownChange,
    bernoulli,
    beta,
    gen,
    pretty,
)
key = jax.random.key(0)
pretty()

In [2]:
@gen
def beta_bernoulli_process(x):
    jax.debug.print("x: {x}", x=x)
    u = x + 1
    jax.debug.print("u: {u}", u=u)
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    z = 2*v
    return {'u': u, 'p': p, 'v': v, 'z': z}

In [3]:
key, subkey = jax.random.split(key)
jitted = jit(beta_bernoulli_process.simulate)
old_trace = jitted(subkey, (1.0,))
old_trace.get_retval()

x: 1.0
u: 2.0


{'p': <jax.Array(0.26836365, dtype=float32)>,
 'u': <jax.Array(2., dtype=float32, weak_type=True)>,
 'v': <jax.Array(0, dtype=int32)>,
 'z': <jax.Array(0, dtype=int32)>}

In [4]:
constraint = C["v"].set(1)
# constraint = C.n()
# arg_diff = (Diff(5.0, UnknownChange),)
arg_diff = (Diff(1.0, NoChange),)

In [5]:
jitted_update = jit(beta_bernoulli_process.update)

key, subkey = jax.random.split(key)
new_trace, weight_diff, ret_diff, discard_choice = jitted_update(
    subkey, old_trace, constraint, arg_diff
)
new_trace.get_retval()

x: 1.0
u: 2.0


{'p': <jax.Array(0.26836365, dtype=float32)>,
 'u': <jax.Array(2., dtype=float32, weak_type=True)>,
 'v': <jax.Array(1, dtype=int32, weak_type=True)>,
 'z': <jax.Array(2, dtype=int32)>}

In [6]:
old_trace.get_choices() == new_trace.get_choices()

True

In [7]:
old_trace.get_choices()

Static({
  'v': Choice(v=<jax.Array(0, dtype=int32)>),
  'p': Choice(v=<jax.Array(0.26836365, dtype=float32)>),
})

In [8]:
new_trace.get_choices()

Static({
  'v': Choice(v=<jax.Array(0, dtype=int32)>),
  'p': Choice(v=<jax.Array(0.26836365, dtype=float32)>),
})

In [9]:
weight_diff, old_trace.get_score() + weight_diff == new_trace.get_score()

(<jax.Array(-0.15127558, dtype=float32)>, <jax.Array(True, dtype=bool)>)

In [10]:
old_trace.get_score()

<jax.Array(-0.4556278, dtype=float32)>

In [11]:
new_trace.get_score()

<jax.Array(-0.6069034, dtype=float32)>